# Transolver
## 1. 模型简介
这是一个用于翼型设计任务的模型，它用于在不同雷诺数和攻角条件下，估计二维翼型周围及表面的物理量。模型参照[Transolver](https://arxiv.org/abs/2402.02366) 中的开源模型代码进行构建。

**模型结构**<br>
主要基于Transformer架构,用Physics-Attention代替了标准的Attention。每个头将输入域编码为一系列物理感知令牌，然后通过令牌之间的注意力来捕获复杂几何形状下的物理相关性。


## 2. 软件环境准备

### 2.1 基于dtk适配软件
<div class="alert alert-warning"> WARNING：镜像中环境已配置，此步骤省略 </div>

### 2.2 软件环境检查


In [1]:
# 检查torch版本
import torch
import os
import onescience

version = torch.__version__
num = float(version[:3])
# assert num == 1.10

# 检查硬件环境
device = "cpu"
if os.system('rocm-smi 2>/dev/null || hy-smi 2>/dev/null')==0:
    device = "dtk"

elif os.system('nvidia-smi 2>/dev/null')==0: 
    device = "cuda"

print("torch version:", version)
print("device =", device)
print("onescience version:", onescience.__version__)


============================ System Management Interface =============================
HCU     Temp     AvgPwr     Perf     PwrCap     VRAM%      HCU%      Mode     
0       48.0C    30.0W      auto     300.0W     0%         0%        N/A      
1       49.0C    23.0W      auto     300.0W     0%         0%        N/A      
2       50.0C    24.0W      auto     300.0W     0%         0%        N/A      
3       50.0C    23.0W      auto     300.0W     0%         0%        N/A      
=================================== End of SMI Log ===================================

torch version: 2.4.1
device = dtk
onescience version: 0.1.0


### 2.2 软件依赖安装


<div class="alert alert-warning"> WARNING: 镜像中环境已配置，此步骤省略 </div>

<div class="alert alert-note" style="color: blue;">
Note: 存在部分文件更新，请卸载当前环境下的 onescience 包，并安装当前目录的 whl 包。
    
请在最顶层目录(包含setup.py文件的目录)，通过命令行，执行下述安装指令：
</div>

In [ ]:
!pip install -e .

## 3. 素材准备
### 3.1 数据集准备

实验数据由[AlfRANS](https://github.com/Extrality/AirfRANS)提供。您可以通过此[链接](https://data.isir.upmc.fr/extrality/NeurIPS_2022/Dataset.zip)直接下载（9.3GB），可以通过此通过此[链接](https://airfrans.readthedocs.io/en/latest/notes/introduction.html)查看数据集的一些描述。


# 数据集结构
```
Dataset/
│
├── .ipynb_checkpoints/
│   # Jupyter Notebook 检查点文件夹
│
├── airFoil2D_SST_<params>/
│   ├── airFoil2D_SST_<params>_aerofoil.vtp
│   │   # 机翼表面网格数据（VTK多边形格式）
│   ├── airFoil2D_SST_<params>_freestream.vtp
│   │   # 流场远场自由流边界的网格数据
│   └── airFoil2D_SST_<params>_internal.vtu
│       # CFD内部计算网格数据，包含速度、压力等物理量
├── airFoil2D_SST_43.597_5.932_3.551_3.1_1.0_18.252
├── ...
```
# 说明

- `<params>` 表示仿真参数编号，包含入口速度、攻角等信息。
- 网格文件用于生成训练数据，通过采样生成点云数据，包括物理场的输入特征和目标值。
- 训练代码会加载这些网格文件，进行采样和预处理，形成PyTorch Geometric支持的数据格式。

模拟名称包含了所有用于生成该模拟的边界条件信息。例如，在 `'airFoil2D_SST_43.597_5.932_3.551_3.1_1.0_18.252'` 中，我们可以读出入口速度大小为 43.597m/s，攻角为5.932°。最后几个数字是NACA翼型的参数，如果有3个参数，表示使用的是4位数系列翼型；如果有4个参数，则说明该翼型来自5位数系列（这是因为最后两位表示翼型厚度，只定义了一个参数）。这里处理的是5位数系列翼型，参数为 \((3.551, 3.1, 1.0, 18.252)\)（它们是实数，不是整数）。 

## 4 训练

### 4.1 单卡训练
运行`python main.py`进行训练

主要参数介绍，可以修改参数更改训练模型的信息：
- data_path：数据集路径，包含所有原始数据文件。默认`./dataset/Dataset`
- save_path：模型及相关指标保存路径。默认`./metrics`
- result_path：预测结果保存路径。默认`./results`
- model：选择训练的模型，可选项包括 Transolver, MLP, GraphSAGE, PointNet, GUNet。默认`Transolver`
- n, --nmodel：训练模型的数量，用于标准差估计。默认值为`1`
- w, --weight：表面损失项前的权重。默认值为`1`
- t, --task：训练任务类型，可选 "full"、"scarce"、"reynolds"、"aoa"。默认`full`
- s, --score：是否在训练模型后直接进行推理评估，0表示不计算，非0表示计算。默认`0`
- gpu：指定使用的GPU编号，默认为0，如无GPU则使用CPU。默认`0`
- n_test：进行推理的翼型数量，将从给定数据集中随机抽取。默认`3`个

In [ ]:
!python main.py --model Transolver -t full --data_path./dataset/Dataset --score0 

### 4.2 多卡训练
运行`mpirun -np <num_GPUs> --allow-run-as-root python main.py`进行训练

主要参数介绍，其他参数参见单卡介绍：
- num_GPUs：卡的数量

In [ ]:
!mpirun -np 4 --allow-run-as-root python main.py

### 4.3 跨节点多卡训练

torchrun启动多节点多卡训练：`torchrun --standalone --nnodes=<num_nodes> --nproc_per_node=<num_GPUs> main.py`
主要参数介绍，其他详细可参考torchrun参数：
- num_nodes： 节点数量
- num_GPUs：卡的数量

In [ ]:
!torchrun --standalone --nnodes=1 --nproc_per_node=4 main.py

## 5 推理

### 5.1 推理
运行`python main_evaluation.py`进行推理，参数同训练的参数一致

In [ ]:
!python main_evaluation.py 